In [1]:
import openai
import pandas as pd
import numpy as np
import re

In [54]:
df = pd.read_excel("../2차결과.xlsx") 
df = df.drop(["review"], axis=1)

df['학생명'] = df['학생명'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', x))
# '제출 시간'을 오름차순으로, '정답 여부'를 내림차순으로 정렬
df = df.sort_values(by=['제출 시간', '정답 여부'], ascending=[True, False])

df

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction
589,송지원,20.05.16 12:29,두정수입력받기,WA,1,"#include <cstdio>\n\nint main(){\nint a,b;\nsc...",Output format mismatch,1
436,송지원,20.05.18 23:24,두정수바꾸어출력하기,AC,NaN,"#include <cstdio>\nint main()\n{\nint a, b;\ns...",NaN,3
588,송지원,20.05.18 23:28,두정수입력받기,AC,NaN,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",NaN,0
1045,송지원,20.05.19 23:22,정수한개입력받아부호바꿔출력하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NaN,2
715,송지원,20.05.20 23:44,시간단위로변환하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NaN,0
...,...,...,...,...,...,...,...,...
812,박유빈,22.12.16 22:10,원의넓이구하기,WA,0,#include<cstdio>\n\nint main()\n{\n int r;\...,type mismatch,0
811,박유빈,22.12.16 22:10,원의넓이구하기,CE,Missingsemicolon,#include<cstdio>\n\nint main()\n{\n int r;\...,NaN,0
813,박유빈,22.12.16 22:11,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,type mismatch,0
814,박유빈,22.12.17 16:08,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,type mismatch,0


In [55]:
# 중복된 문제 이름 제외하고 나머지 제거
new_df = df.drop_duplicates(subset=['학생명', '문제 이름'], keep='first')

new_df = new_df.drop(["정답 여부", "오류 분류", "코드", "대분류", "prediction"], axis=1)

# 인덱스 재설정
new_df = new_df.reset_index(drop=True)

new_df


,학생명,제출 시간,문제 이름
0,송지원,20.05.16 12:29,두정수입력받기
1,송지원,20.05.18 23:24,두정수바꾸어출력하기
2,송지원,20.05.19 23:22,정수한개입력받아부호바꿔출력하기
3,송지원,20.05.20 23:44,시간단위로변환하기
4,송지원,20.05.21 00:06,사칙연산
...,...,...,...
719,한재우,22.12.04 12:07,"초를분,초단위로변환하기"
720,한재우,22.12.04 12:30,내나이가어때서
721,박유빈,22.12.10 17:49,10보다작을까?-1
722,박유빈,22.12.10 17:50,10보다작을까?-2


In [56]:
# 문제 순서 열 추가
new_df['문제 순서'] = new_df.groupby('학생명')['제출 시간'].rank(method='first')
new_df['문제 순서'] = new_df['문제 순서'].astype(int)
new_df

,학생명,제출 시간,문제 이름,문제 순서
0,송지원,20.05.16 12:29,두정수입력받기,1
1,송지원,20.05.18 23:24,두정수바꾸어출력하기,2
2,송지원,20.05.19 23:22,정수한개입력받아부호바꿔출력하기,3
3,송지원,20.05.20 23:44,시간단위로변환하기,4
4,송지원,20.05.21 00:06,사칙연산,5
...,...,...,...,...
719,한재우,22.12.04 12:07,"초를분,초단위로변환하기",7
720,한재우,22.12.04 12:30,내나이가어때서,8
721,박유빈,22.12.10 17:49,10보다작을까?-1,11
722,박유빈,22.12.10 17:50,10보다작을까?-2,12


In [57]:
# 제출 횟수 열 추가
df['제출 횟수'] = df.groupby(['학생명', '문제 이름'])['문제 이름'].transform('count')

new_df2 = df.drop_duplicates(subset=['학생명', '문제 이름'], keep='first')

new_df2 = new_df2.reset_index(drop=True)

new_df2

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,제출 횟수
0,송지원,20.05.16 12:29,두정수입력받기,WA,1,"#include <cstdio>\n\nint main(){\nint a,b;\nsc...",Output format mismatch,1,2
1,송지원,20.05.18 23:24,두정수바꾸어출력하기,AC,NaN,"#include <cstdio>\nint main()\n{\nint a, b;\ns...",NaN,3,1
2,송지원,20.05.19 23:22,정수한개입력받아부호바꿔출력하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NaN,2,1
3,송지원,20.05.20 23:44,시간단위로변환하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NaN,0,1
4,송지원,20.05.21 00:06,사칙연산,AC,NaN,"#include <cstdio> \nint main(){\nint a,b;\nsca...",NaN,2,1
...,...,...,...,...,...,...,...,...,...
719,한재우,22.12.04 12:07,"초를분,초단위로변환하기",CE,Incorrectformatspecifierusage,"#include <cstdio>\n\nint main()\n{\n int a,...",NaN,0,11
720,한재우,22.12.04 12:30,내나이가어때서,AC,NaN,#include <cstdio>\n \nint main()\n{\n int a...,NaN,3,1
721,박유빈,22.12.10 17:49,10보다작을까?-1,AC,NaN,#include<cstdio>\nint main()\n{\n int n;\n ...,NaN,1,1
722,박유빈,22.12.10 17:50,10보다작을까?-2,AC,NaN,#include<cstdio>\nint main()\n{\n int n;\n ...,NaN,4,1


In [58]:
new_df['제출 횟수'] = new_df2['제출 횟수']

new_df.sort_values("학생명", )

new_df = new_df.sort_values(by=['학생명', '제출 시간'], ascending=[True, True])

new_df

,학생명,제출 시간,문제 이름,문제 순서,제출 횟수
203,강석준,21.05.23 16:19,두정수입력받기,1,7
211,강석준,21.05.23 16:42,두정수바꾸어출력하기,2,1
213,강석준,21.05.23 17:07,정수한개입력받아부호바꿔출력하기,3,2
216,강석준,21.05.23 17:10,시간단위로변환하기,4,7
218,강석준,21.05.23 17:15,사칙연산,5,8
...,...,...,...,...,...
574,홍지형,22.04.03 20:41,10보다작을까?-1,10,2
575,홍지형,22.04.03 20:44,10보다작을까?-2,11,1
576,홍지형,22.04.03 20:46,두수비교하기(==),12,1
577,홍지형,22.04.03 20:49,두수비교하기(<=),13,2


In [59]:
new_df.to_excel('../학생별문제풀이순서및시도횟수.xlsx', index=False, engine='openpyxl')

예측

In [15]:
# 제출 횟수 열 추가
df['제출 횟수'] = df.groupby(['학생명', '문제 이름']).cumcount() + 1

# 결과 출력
column_order = ['학생명', '제출 시간','문제 이름', '제출 횟수' ,'정답 여부', '오류 분류', '코드', '대분류', 'prediction' ]
df = df[column_order]

df

,학생명,제출 시간,제출 횟수,문제 이름,정답 여부,오류 분류,코드,대분류,prediction
589,송지원,20.05.16 12:29,1,두정수입력받기,WA,1,"#include <cstdio>\n\nint main(){\nint a,b;\nsc...",output format mismatch,1
436,송지원,20.05.18 23:24,1,두정수바꾸어출력하기,AC,NaN,"#include <cstdio>\nint main()\n{\nint a, b;\ns...",NOT,3
588,송지원,20.05.18 23:28,2,두정수입력받기,AC,NaN,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",NOT,0
1045,송지원,20.05.19 23:22,1,정수한개입력받아부호바꿔출력하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NOT,2
715,송지원,20.05.20 23:44,1,시간단위로변환하기,AC,NaN,#include <cstdio>\nint main()\n{\nint a;\nscan...,NOT,0
...,...,...,...,...,...,...,...,...,...
812,박유빈,22.12.16 22:10,1,원의넓이구하기,WA,0,#include<cstdio>\n\nint main()\n{\n int r;\...,type mismatch,0
811,박유빈,22.12.16 22:10,2,원의넓이구하기,CE,Missing semicolon,#include<cstdio>\n\nint main()\n{\n int r;\...,NOT,0
813,박유빈,22.12.16 22:11,3,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,type mismatch,0
814,박유빈,22.12.17 16:08,4,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,type mismatch,0


In [110]:
df.to_excel('../3차결과.xlsx', index=False, engine='openpyxl')